In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Coursework/Fall\ 2021/682/Project/Dataset/preprocessed_images

/content/drive/MyDrive/Coursework/Fall 2021/682/Project/Dataset/preprocessed_images


In [4]:
%ls

In [19]:
import numpy as np
import pandas as pd
import os
import tqdm
import cv2
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

In [27]:
data_folder = "/content/drive/MyDrive/Coursework/Fall 2021/682/Project/Dataset/"
PAD = True
WIDTH = 512
HEIGHT = 512

In [28]:
train_df = pd.read_csv(f"{data_folder}train.csv")

In [29]:
train_df.head(10)

,image,chain,hotel_id,timestamp
0,8000d59f6a3d765f.jpg,85,24607,2018-11-28 05:42:31
1,8000febe81817f7f.jpg,6,25389,2017-01-01 07:01:46
2,80053befc2d42fba.jpg,3,672,2018-01-26 13:22:07
3,8007fa0a8fe89f9d.jpg,5,63866,2020-07-03 02:31:31
4,800a4a6b6f6d2df6.jpg,0,12036,2017-09-22 17:59:22
5,800a7ffa85c57a3a.jpg,3,25583,2018-11-26 20:44:26
6,800bafacb4555b2f.jpg,82,6261,2016-12-18 16:58:59
7,800e7dd55ffa4056.jpg,2,60181,2018-06-07 20:38:43
8,8011ffce8971daa3.jpg,87,19657,2017-05-27 00:04:06
9,80196e6999ce63cf.jpg,90,55532,2020-06-16 02:46:59


In [37]:
train_df = train_df[train_df['chain'].isin([1,2])]

In [43]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(row_df):
    #print(f"{data_folder}train_images/{row_df.chain.astype(int)}/{row_df.image}")
    img = cv2.imread(f"{data_folder}train_images/{row_df.chain.astype(int)}/{row_df.image}")
    #print(np.shape(img))
    if img is None:
      return None
    if PAD:
        img = pad_image(img)    
    return cv2.resize(img, (WIDTH, HEIGHT))


def save_image(row_df, img):
    #print(f"{data_folder}preprocessed_images/{row_df.image}")
    cv2.imwrite(f"{row_df.image}", img)
    
    
def process_row(row_df):
    #print(row_df)
    img = open_and_preprocess_image(row_df)
    if img is None:
      return -1
    save_image(row_df, img)

In [44]:
%%time
dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_row)(train_df.iloc[i]) for i in range(0, len(train_df)))

CPU times: user 5min 49s, sys: 14.9 s, total: 6min 4s
Wall time: 7min 8s
